<a href="https://colab.research.google.com/github/sauravdani/optimizing-time-and-cost-for-EVs-and-using-Multimodal-transport/blob/main/Multimodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-search-results
!pip install gmplot
!pip install ipython
!pip install folium
!pip install osmnx
!pip install networkx
!pip install Nominatim
!pip install requests


  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=246d753b8ea35fb67f987f6475fd0830a840e8a0d92bc2756f2ae2e684409dbe
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 2.8 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Nominatim: filename=nominatim-0.1-py3-none-any.whl size=2345 sha256=712cf1d29639fe5d9fd2ba3c64b07d8a79b2c02300f704a476ef4338253068f8
  Stored in directory: /root/.cache/pip/wheels/f8/07/74/000f7fd9ad62e9835eb5c224cef28c0aceb3b25c1a8c8a9891
Successfully built Nominatim


In [2]:
from serpapi import GoogleSearch

params = {
  "engine": "google_maps",
  "q": "pizza",
  "ll": "@40.7455096,-74.0083012,15.1z",
  "type": "search",
  "api_key": "bc6d8a1c09a9cf011f1ba830c5a26c6289944148aeee03cac5ac4f36afa53897"
}

search = GoogleSearch(params)
results = search.get_dict()
local_results = results["local_results"]

In [3]:
params = {
  "engine": "google",
  "q": "Fresh Bagels",
  "location": "Seattle-Tacoma, WA, Washington, United States",
  "hl": "en",
  "gl": "us",
  "google_domain": "google.com",
  "num": "10",
  "start": "10",
  "safe": "active",
  "api_key": "bc6d8a1c09a9cf011f1ba830c5a26c6289944148aeee03cac5ac4f36afa53897"
}

search = GoogleSearch(params)
results = search.get_dict()
organic_results = results["organic_results"]

In [4]:
GoogleSearch.SERP_API_KEY = "bc6d8a1c09a9cf011f1ba830c5a26c6289944148aeee03cac5ac4f36afa53897"

In [5]:
query = GoogleSearch({"q": "coffee", "serp_api_key": "Your Private Key"})

In [6]:
from serpapi import GoogleSearch
search = GoogleSearch({})
location_list = search.get_location("delhi", 10)
print(location_list)

[{'id': '585069bdee19ad271e9bc1fc', 'google_id': 20456, 'google_parent_id': 2356, 'name': 'Delhi', 'canonical_name': 'Delhi,India', 'country_code': 'IN', 'target_type': 'Union Territory', 'reach': 32100000, 'gps': [77.10249019999999, 28.7040592], 'keys': ['delhi', 'india']}, {'id': '660725ba914a81973742c144', 'google_id': 9186272, 'google_parent_id': 20456, 'name': 'Delhi Division', 'canonical_name': 'Delhi Division,Delhi,India', 'country_code': 'IN', 'target_type': 'Department', 'gps': [77.10249019999999, 28.7040592], 'reach': 32100000, 'keys': ['delhi', 'division', 'delhi', 'india']}, {'id': '660725b2914a819737416db3', 'google_id': 9075215, 'google_parent_id': 20456, 'name': 'Delhi', 'canonical_name': 'Delhi,Delhi,India', 'country_code': 'IN', 'target_type': 'City', 'gps': [77.10249019999999, 28.7040592], 'reach': 31000000, 'keys': ['delhi', 'delhi', 'india']}, {'id': '585069a8ee19ad271e9b64d6', 'google_id': 1007751, 'google_parent_id': 20456, 'name': 'New Delhi', 'canonical_name': '

In [15]:
import requests
import folium
import osmnx as ox
import networkx as nx

def get_charging_stations_data(api_key, latitude, longitude, distance=50, max_results=100):

    # Open Charge Map API endpoint
    url = "https://api.openchargemap.io/v3/poi/"


    # Parameters for the API call
    params = {
        'output': 'json',
        'latitude': latitude,
        'longitude': longitude,
        'distance': distance,  # Distance in kilometers
        'maxresults': max_results,
        'key': api_key,  # Your Open Charge Map API key
        'countrycode': 'US',  # Restrict to United States
        'verbose': 'true',  # Include detailed information
    }

    # Make the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()

        # Extract required information
        stations_data = []
        for station in data:
            gps = (station['AddressInfo']['Latitude'], station['AddressInfo']['Longitude'])
            power_output = [conn['PowerKW'] for conn in station['Connections']]
            connector_type = [conn['ConnectionType']['Title'] for conn in station['Connections']]
            #availability = station.get('StatusType', {}).get('IsOperational', False)  # Operational status
            pricing = station.get('UsageCost', 'Not available')

            station_info = {
                'gps': gps,
                'power_output': power_output,
                'connector_type': connector_type,
                #'availability': availability,
                'pricing': pricing
            }

            stations_data.append(station_info)

        return stations_data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return []



# Example usage:
api_key = '7a3f7e32-e4be-42b3-8b99-08d20999a056'  # Replace with your Open Charge Map API key
latitude = 37.7749  # Latitude for San Francisco, example center point
longitude = -122.4194  # Longitude for San Francisco, example center point
stations = get_charging_stations_data(api_key, latitude, longitude)

# Display the fetched data
for i, station in enumerate(stations, start=1):
    print(f"Station {i}:")
    print(f"  GPS: {station['gps']}")
    print(f"  Power Output: {station['power_output']} kW")
    print(f"  Connector Type: {station['connector_type']}")
    #print(f"  Availability: {'Available' if station['availability'] else 'Not Available'}")
    print(f"  Pricing: {station['pricing']}")
    print()

Station 1:
  GPS: (37.774142, -122.418547)
  Power Output: [3.7] kW
  Connector Type: ['Type 1 (J1772)']
  Pricing: None

Station 2:
  GPS: (37.775745, -122.418191)
  Power Output: [50, 50] kW
  Connector Type: ['CHAdeMO', 'CCS (Type 1)']
  Pricing: None

Station 3:
  GPS: (37.774933, -122.41726)
  Power Output: [3.7] kW
  Connector Type: ['Type 1 (J1772)']
  Pricing: None

Station 4:
  GPS: (37.774846, -122.417113)
  Power Output: [3.7] kW
  Connector Type: ['Type 1 (J1772)']
  Pricing: None

Station 5:
  GPS: (37.7767355, -122.4189577)
  Power Output: [3.7] kW
  Connector Type: ['Type 1 (J1772)']
  Pricing: None

Station 6:
  GPS: (37.775819, -122.417239)
  Power Output: [3.7] kW
  Connector Type: ['Type 1 (J1772)']
  Pricing: None

Station 7:
  GPS: (37.7772738, -122.4192281)
  Power Output: [3.7] kW
  Connector Type: ['Type 1 (J1772)']
  Pricing: None

Station 8:
  GPS: (37.774966, -122.416038)
  Power Output: [3.7] kW
  Connector Type: ['Type 1 (J1772)']
  Pricing: None

Station 9

In [18]:
import requests
import folium

def get_charging_stations_data(api_key, latitude, longitude, distance=50, max_results=100):
    # Open Charge Map API endpoint
    url = "https://api.openchargemap.io/v3/poi/"

    # Parameters for the API call
    params = {
        'output': 'json',
        'latitude': latitude,
        'longitude': longitude,
        'distance': distance,  # Distance in kilometers
        'maxresults': max_results,
        'key': api_key,  # Your Open Charge Map API key
        'countrycode': 'US',  # Restrict to United States
        'verbose': 'true',  # Include detailed information
    }

    # Make the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()

        # Extract required information
        stations_data = []
        for station in data:
            gps = (station['AddressInfo']['Latitude'], station['AddressInfo']['Longitude'])
            power_output = [conn['PowerKW'] for conn in station['Connections']]
            connector_type = [conn['ConnectionType']['Title'] for conn in station['Connections']]
            pricing = station.get('UsageCost', 'Not available')

            station_info = {
                'gps': gps,
                'power_output': power_output,
                'connector_type': connector_type,
                'pricing': pricing
            }

            stations_data.append(station_info)

        return stations_data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return []

# Example usage:
api_key = '7a3f7e32-e4be-42b3-8b99-08d20999a056'  # Replace with your Open Charge Map API key
latitude = 37.7749  # Latitude for San Francisco, example center point
longitude = -122.4194  # Longitude for San Francisco, example center point
stations = get_charging_stations_data(api_key, latitude, longitude)

# Create the folium map centered on the first station location
m = folium.Map(location=[latitude, longitude], zoom_start=13)

# Add markers for each charging station location
for station in stations:
    folium.Marker(
        location=station['gps'],
        popup=f"Power Output: {station['power_output']} kW\nConnector Type: {station['connector_type']}\nPricing: {station['pricing']}",
        icon=folium.Icon(color='blue')
    ).add_to(m)

# Display the map
m


In [ ]:
import requests
import folium
import osmnx as ox
from shapely.geometry import LineString, Point

def get_charging_stations_data(api_key, latitude, longitude, distance=50, max_results=100):
    # Open Charge Map API endpoint
    url = "https://api.openchargemap.io/v3/poi/"

    # Parameters for the API call
    params = {
        'output': 'json',
        'latitude': latitude,
        'longitude': longitude,
        'distance': distance,  # Distance in kilometers
        'maxresults': max_results,
        'key': api_key,  # Your Open Charge Map API key
        'countrycode': 'US',  # Restrict to United States
        'verbose': 'true',  # Include detailed information
    }

    # Make the API request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()

        # Extract required information
        stations_data = []
        for station in data:
            gps = (station['AddressInfo']['Latitude'], station['AddressInfo']['Longitude'])
            power_output = [conn['PowerKW'] for conn in station['Connections']]
            connector_type = [conn['ConnectionType']['Title'] for conn in station['Connections']]
            pricing = station.get('UsageCost', 'Not available')

            station_info = {
                'gps': gps,
                'power_output': power_output,
                'connector_type': connector_type,
                'pricing': pricing
            }

            stations_data.append(station_info)

        return stations_data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return []

def find_stations_along_route(api_key, route, buffer_distance_km):
    """
    Find all charging stations along a given route within a certain buffer distance.

    Parameters:
    - api_key: Open Charge Map API key
    - route: LineString object representing the route
    - buffer_distance_km: Buffer distance in kilometers to search for charging stations around the route

    Returns:
    - List of charging stations along the route
    """
    stations_along_route = []
    buffer_distance_m = buffer_distance_km * 1000  # Convert km to meters

    # Get bounding box for the route buffer to limit the API request area
    min_lon, min_lat, max_lon, max_lat = route.buffer(buffer_distance_m).bounds

    # Center point of bounding box for API request
    center_lat = (min_lat + max_lat) / 2
    center_lon = (min_lon + max_lon) / 2

    # Fetch stations data around the center of the route's bounding box
    stations = get_charging_stations_data(api_key, center_lat, center_lon, distance=buffer_distance_km)

    # Filter stations to only include those within the buffer area of the route
    for station in stations:
        point = Point(station['gps'][1], station['gps'][0])  # (longitude, latitude)
        if route.buffer(buffer_distance_m).contains(point):
            stations_along_route.append(station)

    return stations_along_route

# Example usage:
api_key = '7a3f7e32-e4be-42b3-8b99-08d20999a056'  # Replace with your Open Charge Map API key

# Define initial and destination locations
start_location = (37.7749, -122.4194)  # San Francisco
end_location = (34.0522, -118.2437)    # Los Angeles

# Create road network graph for routing
G = ox.graph_from_point(start_location, dist=500000, network_type='drive')  # Approx 500 km distance for large area

# Find the nearest nodes in the graph to the start and end locations
start_node = ox.nearest_nodes(G, start_location[1], start_location[0])
end_node = ox.nearest_nodes(G, end_location[1], end_location[0])

# Get the shortest path between start and end nodes using Dijkstra's algorithm
route = nx.shortest_path(G, start_node, end_node, weight='length')

# Convert route nodes to geographic coordinates
route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
route_line = LineString(route_coords)

# Find charging stations along the route
stations_along_route = find_stations_along_route(api_key, route_line, buffer_distance_km=5)

# Create the folium map centered on the start location
m = folium.Map(location=start_location, zoom_start=6)

# Add markers for each charging station along the route
for station in stations_along_route:
    folium.Marker(
        location=station['gps'],
        popup=f"Power Output: {station['power_output']} kW\nConnector Type: {station['connector_type']}\nPricing: {station['pricing']}",
        icon=folium.Icon(color='blue')
    ).add_to(m)

# Optionally, add the route to the map
folium.PolyLine(locations=route_coords, color='green', weight=5).add_to(m)

# Display the map
m


/usr/local/lib/python3.10/dist-packages/osmnx/graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(
/usr/local/lib/python3.10/dist-packages/osmnx/_overpass.py:254: UserWarning: This area is 400 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
